# Algoritmo Breadth-First

El algoritmo de Breadth-First Search expande los nodos del aŕbol de búsqueda de modo jerárquico. , comenzando por el nodo raíz, expandiendo después sus hijos y así sucesivamente.

### Construcción del problema de búsqueda

Definimos un problema basado en el mundo de la aspiradora:

In [1]:
from itertools import product
#Importamos métodos
from VaccuumWorldSearchProblem import State, Transition, SearchProblem

Creamos el modelo de transición a partir de los estados posibles:

In [2]:
cuadros = ['A','B']

#Inicializa contador de estados
i = 0
#Guarda estados
states = {}
#Determina los estados de suciedad
for dirt in product([1,0],repeat=len(cuadros)):
    #Determina la posición del agente
    for agent_loc in cuadros:
        #Asigna valores al objeto estado
        state = State()
        state.world = {cuadros[j]:dirt[j] for j in range(len(cuadros))}
        state.agent_location = agent_loc
        #Guarda el estado en diccionario
        states[i] = state
        #Aumenta el valor de i
        i += 1

Definimos el problema:

In [3]:
#Definimos transiciones
transition_model = Transition()
transition_model.get_transitions(states)

#Definimos el problema de búsqueda
vacuum_problem = SearchProblem(states=states,goal=[6,7], initial=0,
                        transition_model=transition_model,actions=['Clean','Left','Right'])

print(vacuum_problem)

States: 
 -[0, 1, 2, 3, 4, 5, 6, 7]
Initial State:
 -0
Final states:
 -[6, 7]
Actions:
 -['Clean', 'Left', 'Right']
Transition model:
 -[(0, 'Clean', 4), (0, 'Right', 1), (1, 'Clean', 3), (1, 'Left', 0), (2, 'Clean', 6), (2, 'Right', 3), (3, 'Left', 2), (4, 'Right', 5), (5, 'Clean', 7), (5, 'Left', 4), (6, 'Right', 7), (7, 'Left', 6)]


## Aplicación del algoritmo Breadth-First Search

El algoritmo de Breadth-First Search se puede ver más simple que el de Best-First Search. En este caso, relajamos las condiciones para agregar los nodos alcanzados. No contamos con una función de costo en sentido estrcito. Por lo tanto, se utiliza una pila FIFO que permite tomar el nodo al tope de la fila en cada iteración.

In [4]:
from SearchTree import FIFOQueue, Node, expand

In [8]:
class FIFOQueue(object):
    """
    Clase de una cola FIFO en los problemas de búsqueda.
    """
    def __init__(self):
        self.queue = []
  
    def __str__(self):
        return ' '.join([str(q) for q in self.queue])
  
    def isEmpty(self):
        """
        Returns
        -------
        empty : bool
            Devuelve el valor True si el queue está vacío
        """
        return self.queue == []
  
    def push(self, element):
        """
        Arguments
        ---------
        element : 
            El elemento que se va agregar al queue
        """
        self.queue.append(element)
  
    def pop(self):
        """
        Elimina el elemento con más valor según un peso f
        y regresa el elemento correspondiente a este valor.
        """
        #Regresa el primer elemento que llegó
        first_element = self.queue[0]
        #Borra este elemento de la cola
        del self.queue[0]
    
        return first_element
    
    def top(self):
        """
        Devuelve el elemento con la función de costo más alta.
        A diferencia de pop, no elimina el elemento de la pila.
        """
        #Encuentra el elemento máximo en base al costo
        max_element = np.argmax([element.cost for element in self.queue])
        #Guarda el elemento máximo
        item = self.queue[max_element]
        
        return item

In [5]:
def BreadthFirstSearch(problem):
    """
    Algoritmo Breadth-First Search para generar el camino más apto
    en el árbol de búsqueda de un problema.
    
    Argumentos
    ----------
    problem : objeto
        Problema de búsquda.
        
    Salida
    ------
    nodes : list
        Lista de los nodos que llevan a la solución.
    """
    #Almacenamiento de nodos
    nodes = []
    #Nodo inicial
    node = Node()
    node.state = problem.initial
    
    #Revisa si el nodo es meta
    if node in problem.goal:
        return nodes
    
    #Frontera con cola de prioridad
    frontier = FIFOQueue()
    frontier.push(node)
    #Nodos alcanzados
    reached = {problem.initial:node}

    #Mientras la frontera no esté vacía
    while frontier.isEmpty() == False:
        #Pop en frontera
        node = frontier.pop()
        #Guarda el nodo en la lista
        nodes.append(node)
        
        #Expande el nodo actual
        for child in expand(problem, node):
            #Guarda el estado del hijo
            state = child.state
            
            #Revisa si el nodo es meta
            if state in problem.goal:
                nodes.append(child)
                return nodes
            
            #Guarda los hijos no alcanzados o que tengan menor costo
            if state not in reached.keys():
                reached[state] = child
                frontier.push(child)
        
    #Si no logra llegar a un nodo final
    #El algoritmo regresa mensaje de error
    return "No se ha logrado llegar a un estado final."

Aplicamos el algoritmo al problema del mundo de la aspiradora:

In [6]:
tree = BreadthFirstSearch(vacuum_problem)
    
#Observamos el resultado del árbol
for node in tree:
    print("Action: {}, {}, cost: {}".format(node.action, states[node.state], node.cost))

Action: None, World: {'A': 1, 'B': 1}, Loc: A, cost: 0
Action: Clean, World: {'A': 0, 'B': 1}, Loc: A, cost: 1
Action: Right, World: {'A': 1, 'B': 1}, Loc: B, cost: 1
Action: Right, World: {'A': 0, 'B': 1}, Loc: B, cost: 2
Action: Clean, World: {'A': 0, 'B': 0}, Loc: B, cost: 3


In [7]:
def get_path(search_tree, root):
    """
    Recorre el árbol de búsqueda de atrás hacia el inicio para obtener el camino de acciones
    que llevaron a la meta.
    """
    #Meta
    leaf = search_tree[::-1][0]
    #Padre de la meta
    parent = leaf.parent
    #Adjunta la meta y su padre al camino
    path = [leaf, parent]
    #Recorre el árbol de búsqueda hasta el inicial
    while parent.state != root:
        parent = parent.parent
        #Adjunta los padres en el camino
        path.append(parent)
        
    return path[::-1]
    
for n in get_path(tree, vacuum_problem.initial):
    print("{} ----->\n{}".format(n.action, states[n.state]))

None ----->
World: {'A': 1, 'B': 1}, Loc: A
Clean ----->
World: {'A': 0, 'B': 1}, Loc: A
Right ----->
World: {'A': 0, 'B': 1}, Loc: B
Clean ----->
World: {'A': 0, 'B': 0}, Loc: B


## Ejercicio

Programar el algoritmo de <b>Depth First Search</b> (primero en profundidad) para el problema del 8-puzzle. Comenzar tomando como estado inicial el estado:

$$[1,e,2] \\ [6,3,4] \\ [7,5,8]$$

Probar otros estados para determinar si el algoritmo es capaz de converger apropiadamente.